In [39]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline


In [40]:
data = pd.read_csv('train.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data) 

data_test = data[0:1000].T
Y_test = data_test[0]
X_test = data_test[1:n]
X_test = X_test / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [41]:
def init_params():
    w1 = np.random.rand(10,784) -0.5
    b1 = np.random.rand(10,1) - 0.5
    w2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,1) - 0.5

    return w1, b1, w2, b2

def ReLu(Z):
    return np.maximum(0,Z)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

def forward_prop(w1,b1,w2,b2,X):
    z1 = np.dot(w1,X) + b1
    a1 = ReLu(z1)
    z2 = np.dot(w2,a1) + b2
    a2 = softmax(z2)
    return z1,a1,z2,a2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def ReLU_deriv(Z):
    return Z > 0
    
def back_prop(z1,a1,z2,a2,w1,w2,X,Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = a2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(a1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = w2.T.dot(dZ2) * ReLU_deriv(z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2



In [42]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i == iterations - 1:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [43]:
w1, b1, w2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  499
[2 3 9 ... 3 5 3] [2 3 9 ... 3 7 2]
0.8368292682926829


In [44]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

In [45]:
test_predictions = make_predictions(X_test, w1, b1, w2, b2)
get_accuracy(test_predictions, Y_test)

[2 7 5 9 3 7 6 1 9 7 6 1 2 9 9 0 7 1 3 8 7 6 0 5 3 3 9 1 8 4 1 5 1 3 8 2 0
 0 5 5 9 3 6 1 3 6 5 9 1 4 6 8 9 7 7 7 0 7 8 9 4 3 9 1 7 6 9 8 1 7 8 1 1 3
 2 3 7 0 3 6 4 2 1 7 9 6 2 8 9 4 3 2 7 0 1 8 5 4 2 7 6 5 4 9 6 0 0 1 7 9 0
 0 7 4 4 6 3 0 3 9 5 6 1 5 3 0 9 5 6 0 3 1 7 7 6 6 8 6 5 3 8 3 1 6 8 5 1 9
 5 1 8 6 7 4 3 1 3 1 7 0 3 5 9 1 3 9 7 8 4 7 7 2 3 4 7 8 9 7 7 3 2 1 1 0 9
 0 0 2 8 9 0 8 2 3 1 9 6 5 7 3 0 1 9 2 2 3 4 8 8 7 7 7 9 6 4 2 5 9 2 7 9 2
 6 6 4 8 2 6 4 8 5 2 5 9 0 8 1 6 2 7 3 5 9 5 1 4 6 3 7 4 6 4 6 4 7 2 0 6 2
 3 1 4 5 9 3 9 7 6 4 2 0 9 0 3 4 8 2 0 6 1 8 8 0 9 3 2 1 2 6 6 4 2 0 1 2 1
 0 7 3 6 8 7 7 3 3 8 1 2 6 7 0 6 0 5 8 5 3 4 1 1 4 7 2 3 2 9 9 8 1 2 9 1 6
 4 4 5 9 0 2 7 0 8 7 3 2 7 6 0 3 1 6 7 1 2 6 2 3 6 6 6 6 5 8 6 3 4 1 6 7 3
 3 0 0 1 3 3 9 7 0 6 0 9 7 0 4 6 1 2 9 4 4 3 4 2 3 1 6 8 8 6 9 5 2 1 0 9 5
 2 5 4 1 9 4 3 7 0 0 8 9 3 9 3 6 7 4 3 2 9 8 6 0 1 8 8 6 9 7 9 4 2 1 7 7 5
 4 0 9 9 5 1 7 4 4 1 3 0 6 3 2 7 1 5 3 4 6 0 8 6 1 9 8 3 1 2 4 3 3 3 7 4 0
 6 0 5 2 2 9 0 3 1 3 3 7 

0.847